In [5]:
from helpers import *
from gbd_core.api import GBD

feat = get_available_features()

with GBD([ '/home/raphael-zipperer/Uni/BA/database/base.db', '/home/raphael-zipperer/Uni/BA/database/meta.db' ]) as gbd:
    data = gbd.query('(track=main_2023 or track=main_2024) and minisat1m!=yes', resolve=feat + ["family"])

df = pd.read_csv('top40.csv')

distinct_configs = df['configuration'].drop_duplicates().tolist()

print(getVirtual(data['hash'].tolist(), distinct_configs))

ranked = []
for c in distinct_configs:
    confs = distinct_configs.copy()
    confs.remove(c)
    ranked.append((c, getVirtual(data['hash'].tolist(), confs)))

ranked.sort(key=lambda x: x[1])



848.9443285276767


In [6]:
from helpers import *
from gbd_core.api import GBD
import pickle


def addconfs(beam, confs, w):
    ranked = []
    for b in beam:
        for c in confs:
            if not c in beam:
                ranked.append((b + [c], getVirtual(data['hash'].tolist(), b + [c])))
    ranked.sort(key=lambda x: x[1])
    return ranked[:w]

                


feat = get_available_features()

with GBD([ '/home/raphael-zipperer/Uni/BA/database/base.db', '/home/raphael-zipperer/Uni/BA/database/meta.db' ]) as gbd:
    data = gbd.query('(track=main_2023 or track=main_2024) and minisat1m!=yes', resolve=feat + ["family"])

df = pd.read_csv('top40.csv')

distinct_configs = df['configuration'].drop_duplicates().tolist()

ranked = []
for c in distinct_configs:
    confs = distinct_configs.copy()
    confs.remove(c)
    ranked.append((c, getVirtual(data['hash'].tolist(), [c])))

ranked.sort(key=lambda x: x[1])
w = 40
print(ranked)

beam = [[x[0]] for x in ranked][:w]
for i in range(3):
    ranked = addconfs(beam, distinct_configs, w)
    print([x[1] for x in ranked])
    beam = [x[0] for x in ranked]
with open('best/best4.pkl', 'wb') as f:
    pickle.dump(beam[0], f)

[('Default', 1224.408516625956), ("{'backbone': 2, 'bump': 1, 'chrono': 0, 'congruence': 0, 'eliminate': 1, 'extract': 1, 'factor': 1, 'fastel': 0, 'forward': 0, 'lucky': 1, 'phase': 0, 'phasesaving': 1, 'preprocess': 1, 'probe': 1, 'randec': 1, 'reluctant': 0, 'reorder': 2, 'rephase': 1, 'restart': 0, 'stable': 2, 'substitute': 0, 'sweep': 1, 'target': 2, 'transitive': 0, 'vivify': 1, 'warmup': 0}", 1484.5995554801505), ("{'backbone': 2, 'bump': 1, 'chrono': 1, 'congruence': 1, 'eliminate': 0, 'extract': 1, 'factor': 1, 'fastel': 0, 'forward': 0, 'lucky': 1, 'phase': 1, 'phasesaving': 1, 'preprocess': 0, 'probe': 1, 'randec': 0, 'reluctant': 1, 'reorder': 2, 'rephase': 1, 'restart': 1, 'stable': 1, 'substitute': 0, 'sweep': 0, 'target': 2, 'transitive': 1, 'vivify': 1, 'warmup': 1}", 1497.1148709245665), ("{'backbone': 2, 'bump': 1, 'chrono': 1, 'congruence': 0, 'eliminate': 1, 'extract': 1, 'factor': 0, 'fastel': 1, 'forward': 0, 'lucky': 1, 'phase': 1, 'phasesaving': 1, 'preprocess'

In [7]:
conf = beam[0]
df = pd.read_csv('../classifier/top40.csv')
df = df.groupby('configuration')['time'].sum()

for c in conf:
    index = df.index.tolist().index(c) if c in df.index else -1
    print(index)

with GBD([ '/home/raphael-zipperer/Uni/BA/database/base.db', '/home/raphael-zipperer/Uni/BA/database/meta.db' ]) as gbd:
    data = gbd.query('(track=main_2023 or track=main_2024) and minisat1m!=yes', resolve=["family"])

df = pd.read_csv('../classifier/top40.csv')

data['best'] = "N/A"

for hash in data['hash'].tolist():
    df = pd.read_csv('../classifier/top40.csv')
    df = df[df['key'] == hash]
    df = df[df['configuration'].isin(conf)]

    if not df.empty:
        min_time_config = df.loc[df['time'].idxmin(), 'configuration']
        print(min_time_config)
        data.loc[data['hash'] == hash, 'best'] = min_time_config



0
25
3
33
{'backbone': 2, 'bump': 1, 'chrono': 1, 'congruence': 0, 'eliminate': 1, 'extract': 1, 'factor': 1, 'fastel': 0, 'forward': 1, 'lucky': 0, 'phase': 0, 'phasesaving': 1, 'preprocess': 0, 'probe': 0, 'randec': 0, 'reluctant': 1, 'reorder': 0, 'rephase': 0, 'restart': 1, 'stable': 2, 'substitute': 0, 'sweep': 0, 'target': 2, 'transitive': 1, 'vivify': 1, 'warmup': 1}
{'backbone': 2, 'bump': 1, 'chrono': 0, 'congruence': 1, 'eliminate': 1, 'extract': 0, 'factor': 0, 'fastel': 1, 'forward': 0, 'lucky': 1, 'phase': 0, 'phasesaving': 1, 'preprocess': 0, 'probe': 1, 'randec': 0, 'reluctant': 0, 'reorder': 2, 'rephase': 1, 'restart': 1, 'stable': 0, 'substitute': 1, 'sweep': 1, 'target': 0, 'transitive': 1, 'vivify': 1, 'warmup': 1}
{'backbone': 0, 'bump': 0, 'chrono': 0, 'congruence': 1, 'eliminate': 1, 'extract': 1, 'factor': 0, 'fastel': 0, 'forward': 1, 'lucky': 0, 'phase': 0, 'phasesaving': 0, 'preprocess': 0, 'probe': 1, 'randec': 1, 'reluctant': 1, 'reorder': 2, 'rephase': 0, '

In [17]:

family_counts = data['family'].value_counts()
families_to_replace = family_counts[family_counts < 5].index

data['family'] = data['family'].apply(lambda x: 'rest' if x in families_to_replace else x)

family_stats = data.groupby('family')['best'].value_counts(normalize=True)

for family, stats in family_stats.groupby(level=0):
    most_common_config = stats.idxmax()[1]
    percentage = stats.max() * 100
    print(f"Family: {family}, Most Common Configuration: {most_common_config}, Percentage: {percentage:.2f}%")

    family_config_relationship = {}

for family, stats in family_stats.groupby(level=0):
    most_common_config = stats.idxmax()[1]
    percentage = stats.max() * 100
    family_config_relationship[family] = {
        "most_common_config": most_common_config,
        "percentage": percentage
    }
print(family_config_relationship)

config_family = {c:[] for c in conf}

for family, stats in family_config_relationship.items():
    config_family[stats["most_common_config"]].append((family, stats["percentage"]))

for c, list in config_family.items():
    list.sort(key=lambda x: x[1], reverse=True)
    config_family[c] = list

for c, l in config_family.items():
    print(c)
    for entry in l:
        print(entry)

Family: argumentation, Most Common Configuration: {'backbone': 2, 'bump': 1, 'chrono': 1, 'congruence': 0, 'eliminate': 1, 'extract': 1, 'factor': 1, 'fastel': 0, 'forward': 1, 'lucky': 0, 'phase': 0, 'phasesaving': 1, 'preprocess': 0, 'probe': 0, 'randec': 0, 'reluctant': 1, 'reorder': 0, 'rephase': 0, 'restart': 1, 'stable': 2, 'substitute': 0, 'sweep': 0, 'target': 2, 'transitive': 1, 'vivify': 1, 'warmup': 1}, Percentage: 53.66%
Family: bitvector, Most Common Configuration: {'backbone': 2, 'bump': 1, 'chrono': 0, 'congruence': 1, 'eliminate': 1, 'extract': 0, 'factor': 0, 'fastel': 1, 'forward': 0, 'lucky': 1, 'phase': 0, 'phasesaving': 1, 'preprocess': 0, 'probe': 1, 'randec': 0, 'reluctant': 0, 'reorder': 2, 'rephase': 1, 'restart': 1, 'stable': 0, 'substitute': 1, 'sweep': 1, 'target': 0, 'transitive': 1, 'vivify': 1, 'warmup': 1}, Percentage: 60.00%
Family: brent-equations, Most Common Configuration: {'backbone': 2, 'bump': 1, 'chrono': 1, 'congruence': 0, 'eliminate': 1, 'extr